**1. Install dependencies**

In [1]:
!pip install -q --upgrade \
  transformers \
  datasets \
  accelerate \
  peft \
  trl \
  bitsandbytes \
  safetensors \
  mlflow \
  dagshub \
  sqlglot \
  pandas \
  tqdm \
  openai \
  anthropic \
  tenacity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 140.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.0/597.0 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 144.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

**2. Basic environment + GPU sanity check**

In [2]:
import os, sys, time, json, math, random
from pathlib import Path

import torch

assert torch.cuda.is_available(), \
    "No GPU detected — change runtime to A100 (Runtime → Change runtime type → GPU)."

print("Python:", sys.version)
print("Torch :", torch.__version__)
print("CUDA  :", torch.version.cuda)
print("GPU   :", torch.cuda.get_device_name(0))
print("Time  :", time.strftime("%Y-%m-%d %H:%M:%S"))

# Reproducibility
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Quick GPU allocation test
x = torch.randn(1024, 1024, device="cuda")
print("GPU tensor OK:", x.shape, x.dtype)
del x
torch.cuda.empty_cache()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch : 2.9.0+cu126
CUDA  : 12.6
GPU   : NVIDIA A100-SXM4-40GB
Time  : 2026-02-06 08:58:59
GPU tensor OK: torch.Size([1024, 1024]) torch.float32


**3. Imports + Global Configuration**

In [3]:
import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

import sqlglot
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type


# Global Configuration

BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"

# Training settings (fast + stable for your budget)
MAX_SEQ_LEN = 1024
EPOCHS = 4
LR = 2e-4
PER_DEVICE_BATCH = 2
GRAD_ACCUM = 8        # effective batch = 16
WARMUP_RATIO = 0.05

# Evaluation / generation
MAX_NEW_TOKENS = 196
BATCH_SIZE_EVAL = 12


**4. DagsHub + MLflow Setup**

In [4]:
import dagshub
import mlflow
import time

# Initialize DagsHub tracking (you already have this repo connected)
dagshub.init(repo_owner="aravula7", repo_name="llm-finetuning", mlflow=True)

# Create / select experiment
mlflow.set_experiment("text2sql_qwen25_3b")

# Create a timestamped parent run name we’ll reuse later
PARENT_RUN_NAME = f"colab_run_{time.strftime('%Y%m%d_%H%M%S')}"

print("DagsHub connected and MLflow experiment ready:")
print("Experiment:", mlflow.get_experiment_by_name("text2sql_qwen25_3b").experiment_id)
print("Parent run name:", PARENT_RUN_NAME)

Accessing as aravula7

Initialized MLflow to track repo "aravula7/llm-finetuning"

Repository aravula7/llm-finetuning initialized!

DagsHub connected and MLflow experiment ready:
Experiment: 1
Parent run name: colab_run_20260206_085921


**5. Load datasets**

In [5]:
from datasets import load_dataset

train_path = "./data/train.jsonl"
val_path   = "./data/validation.jsonl"
test_path  = "./data/test.jsonl"

data_files = {
    "train": train_path,
    "validation": val_path,
    "test": test_path,
}

ds = load_dataset("json", data_files=data_files)

print("Dataset loaded successfully.")
print("Split sizes:", {k: len(ds[k]) for k in ds})
print("Columns:", ds["train"].column_names)

# Quick sanity preview
print("\nSample train example keys:")
print(ds["train"][0].keys())

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset loaded successfully.
Split sizes: {'train': 350, 'validation': 50, 'test': 100}
Columns: ['instruction', 'schema_context', 'input', 'output']

Sample train example keys:
dict_keys(['instruction', 'schema_context', 'input', 'output'])


**6. Build SQL-only prompts + apply Qwen chat template**

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# Make batching safer
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

SQL_ONLY_RULE = (
    "Return ONLY the PostgreSQL query. "
    "Do NOT include explanations, markdown, code fences, or commentary."
)

def build_user_prompt(example):
    """
    Deterministic prompt builder used for:
    - training
    - validation
    - evaluation
    """
    return (
        f"{example['instruction'].strip()}\n\n"
        f"{SQL_ONLY_RULE}\n\n"
        f"Schema:\n{example['schema_context'].strip()}\n\n"
        f"Request:\n{example['input'].strip()}\n"
    )

def format_for_sft(example):
    """
    Convert each row into a single chat-formatted training string
    compatible with Qwen2.5-Instruct.
    """
    user_prompt = build_user_prompt(example)
    assistant_answer = example["output"].strip()

    messages = [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant_answer},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    return {"text": text}

# Apply formatting
train_sft = ds["train"].map(format_for_sft, remove_columns=ds["train"].column_names)
val_sft   = ds["validation"].map(format_for_sft, remove_columns=ds["validation"].column_names)

print("Formatted for SFT.")
print("Example training text (truncated):\n")
print(train_sft[0]["text"][:600], "...\n")
print("Token count of this example:",
      len(tokenizer(train_sft[0]['text'], add_special_tokens=False).input_ids))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Formatted for SFT.
Example training text (truncated):

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Generate a PostgreSQL query for the following request using the provided schema.

Return ONLY the PostgreSQL query. Do NOT include explanations, markdown, code fences, or commentary.

Schema:
Table: customers (customer_id, email, full_name, state)

Request:
List customer_id, email, and full_name for customers in state CA.
<|im_end|>
<|im_start|>assistant
SELECT customer_id, email, full_name FROM customers WHERE state = 'CA' ORDER BY customer_id;<|im_end|>
 ...

Token count of this example: 117


**7. Token length sanity checks** *(to detect truncation risk before training)*

In [7]:
import numpy as np

def token_len_text(s: str) -> int:
    return len(tokenizer(s, add_special_tokens=False).input_ids)

def length_stats(dataset, n=200, seed=SEED):
    n = min(n, len(dataset))
    sample = dataset.shuffle(seed=seed).select(range(n))
    lens = [token_len_text(x["text"]) for x in sample]
    lens_sorted = sorted(lens)
    p95 = lens_sorted[int(0.95 * len(lens_sorted)) - 1]
    p99 = lens_sorted[int(0.99 * len(lens_sorted)) - 1]
    return {
        "n_sampled": n,
        "min": int(min(lens)),
        "mean": float(np.mean(lens)),
        "p95": int(p95),
        "p99": int(p99),
        "max": int(max(lens)),
        "max_seq_len": MAX_SEQ_LEN,
        "pct_over_max_seq": float(sum(l > MAX_SEQ_LEN for l in lens) / len(lens)),
    }

train_stats = length_stats(train_sft, n=350)
val_stats   = length_stats(val_sft, n=50)

print("Train token-length stats:", train_stats)
print("Val   token-length stats:", val_stats)

if train_stats["pct_over_max_seq"] > 0 or val_stats["pct_over_max_seq"] > 0:
    print("\n Some examples exceed MAX_SEQ_LEN and will be truncated. Consider increasing MAX_SEQ_LEN or shortening schema_context.")
else:
    print("\n No truncation expected with current MAX_SEQ_LEN.")

Train token-length stats: {'n_sampled': 350, 'min': 104, 'mean': 183.54, 'p95': 297, 'p99': 354, 'max': 430, 'max_seq_len': 1024, 'pct_over_max_seq': 0.0}
Val   token-length stats: {'n_sampled': 50, 'min': 112, 'mean': 168.28, 'p95': 221, 'p99': 243, 'max': 261, 'max_seq_len': 1024, 'pct_over_max_seq': 0.0}

 No truncation expected with current MAX_SEQ_LEN.


**8. Load Qwen in 4-bit for QLoRA training**

In [8]:
# Cell 8: Fix bitsandbytes for 4-bit QLoRA

!pip install -q --upgrade bitsandbytes

import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)
print("✅ bitsandbytes upgraded. If it still errors on reload, restart runtime and re-run from imports.")

bitsandbytes version: 0.49.1
✅ bitsandbytes upgraded. If it still errors on reload, restart runtime and re-run from imports.


In [9]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_cfg_train = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg_train,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Required for gradient checkpointing + LoRA stability
model_4bit.config.use_cache = False

print("Loaded Qwen 2.5-3B-Instruct in 4-bit for QLoRA.")
print("Trainable params BEFORE LoRA:", sum(p.numel() for p in model_4bit.parameters() if p.requires_grad))

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded Qwen 2.5-3B-Instruct in 4-bit for QLoRA.
Trainable params BEFORE LoRA: 311406592


**9. LoRA config + SFTConfig (QLoRA training setup)**

In [14]:
from peft import LoraConfig
from trl import SFTConfig

# LoRA configuration (solid default for 3B text-to-SQL)
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

# Replace deprecated warmup_ratio with warmup_steps (5% of total update steps)
total_train_steps = math.ceil(len(train_sft) / PER_DEVICE_BATCH / GRAD_ACCUM) * EPOCHS
warmup_steps = max(1, int(0.05 * total_train_steps))

sft_cfg = SFTConfig(
    output_dir="./outputs",
    num_train_epochs=EPOCHS,
    learning_rate=LR,

    per_device_train_batch_size=PER_DEVICE_BATCH,
    gradient_accumulation_steps=GRAD_ACCUM,

    warmup_steps=warmup_steps,
    lr_scheduler_type="cosine",

    logging_steps=10,
    eval_steps=50,
    save_steps=200,
    save_total_limit=2,

    packing=True,                 # you chose YES
    gradient_checkpointing=True,  # saves memory

    bf16=True,                    # best on A100
    optim="paged_adamw_8bit",     # fast + memory efficient
    report_to=[],                 # we'll use MLflow manually
)

print("LoRA + SFTConfig ready.")
print("Total train steps:", total_train_steps, "| Warmup steps:", warmup_steps)
print("LoRA target modules:", lora_cfg.target_modules)

LoRA + SFTConfig ready.
Total train steps: 88 | Warmup steps: 4
LoRA target modules: {'down_proj', 'up_proj', 'q_proj', 'k_proj', 'o_proj', 'gate_proj', 'v_proj'}


**10. Create SFTTrainer + start MLflow parent run + train**

In [21]:
from trl import SFTTrainer
import mlflow
import os

# KV cache off during training
model_4bit.config.use_cache = False

with mlflow.start_run(run_name=PARENT_RUN_NAME) as parent_run:
    mlflow.set_tag("project", "text2sql")
    mlflow.set_tag("base_model", BASE_MODEL)
    mlflow.set_tag("train_mode", "QLoRA_4bit")
    mlflow.log_params({
        "max_seq_len": MAX_SEQ_LEN,
        "epochs": EPOCHS,
        "lr": LR,
        "per_device_batch": PER_DEVICE_BATCH,
        "grad_accum": GRAD_ACCUM,
        "effective_batch": PER_DEVICE_BATCH * GRAD_ACCUM,
        "warmup_steps": warmup_steps,
        "packing": True,
        "optimizer": "paged_adamw_8bit",
        "dtype": "bf16",
        "lora_r": lora_cfg.r,
        "lora_alpha": lora_cfg.lora_alpha,
        "lora_dropout": lora_cfg.lora_dropout,
        "lora_targets": ",".join(sorted(list(lora_cfg.target_modules))),
    })

    with mlflow.start_run(run_name="train_qlora", nested=True):
        trainer = SFTTrainer(
            model=model_4bit,
            args=sft_cfg,
            train_dataset=train_sft,
            eval_dataset=val_sft,
            peft_config=lora_cfg
        )

        train_result = trainer.train()
        trainer.save_model("./finetuned_qwen_lora")

        # Training metrics
        metrics = {k: float(v) for k, v in (train_result.metrics or {}).items()}
        if metrics:
            mlflow.log_metrics(metrics)

        # Validation metrics at end (useful in DagsHub UI)
        eval_metrics = trainer.evaluate()
        eval_metrics = {k: float(v) for k, v in (eval_metrics or {}).items()}
        if eval_metrics:
            mlflow.log_metrics({f"final_{k}": v for k, v in eval_metrics.items()})

        print("Training complete. Adapter saved to ./finetuned_qwen_lora")
        print("Final eval metrics:", eval_metrics)

Step,Training Loss
10,1.718970
20,0.388982


Training complete. Adapter saved to ./finetuned_qwen_lora
Final eval metrics: {'eval_loss': 0.3354441225528717, 'eval_runtime': 0.9909, 'eval_samples_per_second': 9.082, 'eval_steps_per_second': 2.018}
🏃 View run train_qlora at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/720d22dd85524124883d5054d43b806b
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1
🏃 View run colab_run_20260206_085921 at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/af57ea077bf74393988b7afe98284a0a
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1


**11. Merge LoRA adapter into a FP16 base model**

In [22]:
import torch
from transformers import AutoModelForCausalLM

ADAPTER_DIR = "./finetuned_qwen_lora"
MERGED_DIR  = "./finetuned_qwen_fp16_merged"

# Load base model in FP16/BF16 (no 4-bit) for clean merge
base_fp16 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    dtype=torch.float16,
)

# Load adapter onto base and merge
merged = PeftModel.from_pretrained(base_fp16, ADAPTER_DIR)
merged = merged.merge_and_unload()

# For inference, enable KV cache
merged.config.use_cache = True

# Save merged model + tokenizer
merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print(" Merged FP16 model saved to:", MERGED_DIR)

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

 Merged FP16 model saved to: ./finetuned_qwen_fp16_merged


**12. Create INT8 inference model from merged FP16 checkpoint**

In [23]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

INT8_DIR = "./finetuned_qwen_int8"

bnb_cfg_int8 = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

model_int8 = AutoModelForCausalLM.from_pretrained(
    "./finetuned_qwen_fp16_merged",
    device_map="auto",
    quantization_config=bnb_cfg_int8,
)

model_int8.config.use_cache = True

# Save as an INT8 loadable checkpoint (config + tokenizer already in merged dir)
model_int8.save_pretrained(INT8_DIR, safe_serialization=True)
tokenizer.save_pretrained(INT8_DIR)

print(" INT8 model checkpoint saved to:", INT8_DIR)

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

 INT8 model checkpoint saved to: ./finetuned_qwen_int8


**13. Evaluation utilities (parseable SQL, exact match, latency)**

In [24]:
import re, time, statistics
import sqlglot

def strip_code_fences(s: str) -> str:
    if s is None:
        return ""
    s = s.strip()
    # remove ```sql ... ``` or ``` ... ```
    s = re.sub(r"^```(?:sql)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)
    # remove leading "SQL:" etc
    s = re.sub(r"^\s*(sql\s*:\s*)", "", s, flags=re.IGNORECASE)
    return s.strip()

def is_parseable_sql(sql: str, dialect="postgres") -> bool:
    sql = strip_code_fences(sql)
    if not sql:
        return False
    try:
        sqlglot.parse_one(sql, read=dialect)
        return True
    except Exception:
        return False

def normalize_sql(sql: str, dialect="postgres") -> str:
    sql = strip_code_fences(sql)
    if not sql:
        return ""
    # Normalize via sqlglot (best-effort)
    try:
        expr = sqlglot.parse_one(sql, read=dialect)
        return expr.sql(dialect=dialect, pretty=False).strip().rstrip(";")
    except Exception:
        # fallback normalization
        return re.sub(r"\s+", " ", sql).strip().rstrip(";")

def exact_match(pred_sql: str, gold_sql: str) -> bool:
    return normalize_sql(pred_sql) == normalize_sql(gold_sql)

def latency_stats(latencies):
    if not latencies:
        return {"mean": None, "p50": None, "p95": None}
    lat_sorted = sorted(latencies)
    mean = sum(lat_sorted) / len(lat_sorted)
    p50 = lat_sorted[int(0.50 * len(lat_sorted)) - 1]
    p95 = lat_sorted[max(0, int(0.95 * len(lat_sorted)) - 1)]
    return {"mean": mean, "p50": p50, "p95": p95}

**14. Load local models + batched greedy inference on test (baseline FP16, finetuned FP16, finetuned INT8)**

In [25]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Use only test split (100 examples)
test_raw = ds["test"]

def build_prompts_from_split(split):
    prompts = []
    gold = []
    for ex in split:
        prompts.append(build_user_prompt(ex))
        gold.append(ex["output"])
    return prompts, gold

test_prompts, test_gold = build_prompts_from_split(test_raw)
print("Test examples:", len(test_prompts))

@torch.inference_mode()
def batched_generate(model, prompts, batch_size=BATCH_SIZE_EVAL, max_new_tokens=MAX_NEW_TOKENS):
    model.eval()
    latencies = []
    outputs = []

    for i in tqdm(range(0, len(prompts), batch_size), desc="Batched inference"):
        batch_prompts = prompts[i:i+batch_size]

        # Build chat messages + apply template with generation prompt
        batch_texts = []
        for p in batch_prompts:
            msgs = [{"role":"user", "content": p}]
            batch_texts.append(
                tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
            )

        enc = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SEQ_LEN,
        ).to(model.device)

        start = time.perf_counter()
        gen = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        end = time.perf_counter()

        # Per-example latency approximation: batch_time / batch_size
        batch_time = end - start
        per_ex = batch_time / len(batch_prompts)
        latencies.extend([per_ex] * len(batch_prompts))

        # Decode only newly generated tokens
        for j in range(len(batch_prompts)):
            prompt_len = enc["input_ids"][j].shape[0]
            decoded = tokenizer.decode(gen[j][prompt_len:], skip_special_tokens=True)
            outputs.append(strip_code_fences(decoded))

    return outputs, latencies

def eval_predictions(preds, gold):
    parseable = [is_parseable_sql(p) for p in preds]
    exact = [exact_match(p, g) for p, g in zip(preds, gold)]
    return {
        "parseable_rate": sum(parseable) / len(parseable),
        "exact_match_rate": sum(exact) / len(exact),
    }

# 1) Baseline (raw) Qwen FP16
baseline_fp16 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    dtype=torch.float16,
)
baseline_fp16.config.use_cache = True

# 2) Fine-tuned merged FP16
finetuned_fp16 = AutoModelForCausalLM.from_pretrained(
    "./finetuned_qwen_fp16_merged",
    device_map="auto",
    dtype=torch.float16,
)
finetuned_fp16.config.use_cache = True

# 3) Fine-tuned INT8
bnb_cfg_int8 = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)
finetuned_int8 = AutoModelForCausalLM.from_pretrained(
    "./finetuned_qwen_int8",
    device_map="auto",
    quantization_config=bnb_cfg_int8,
)
finetuned_int8.config.use_cache = True

# Run eval for each local model
local_results = {}

pred_base, lat_base = batched_generate(baseline_fp16, test_prompts)
m_base = eval_predictions(pred_base, test_gold)
local_results["qwen_baseline_fp16"] = {**m_base, **{f"lat_{k}": v for k, v in latency_stats(lat_base).items()}}

pred_ft, lat_ft = batched_generate(finetuned_fp16, test_prompts)
m_ft = eval_predictions(pred_ft, test_gold)
local_results["qwen_finetuned_fp16"] = {**m_ft, **{f"lat_{k}": v for k, v in latency_stats(lat_ft).items()}}

pred_i8, lat_i8 = batched_generate(finetuned_int8, test_prompts)
m_i8 = eval_predictions(pred_i8, test_gold)
local_results["qwen_finetuned_int8"] = {**m_i8, **{f"lat_{k}": v for k, v in latency_stats(lat_i8).items()}}

local_results

Test examples: 100


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:246: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Batched inference:   0%|          | 0/9 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Batched inference:   0%|          | 0/9 [00:00<?, ?it/s]

Batched inference:   0%|          | 0/9 [00:00<?, ?it/s]

{'qwen_baseline_fp16': {'parseable_rate': 1.0,
  'exact_match_rate': 0.09,
  'lat_mean': 0.40546880933996365,
  'lat_p50': 0.42239852066662326,
  'lat_p95': 0.6238893242499822},
 'qwen_finetuned_fp16': {'parseable_rate': 0.93,
  'exact_match_rate': 0.13,
  'lat_mean': 0.5267278591099966,
  'lat_p50': 0.7107913513333793,
  'lat_p95': 0.7394641423332663},
 'qwen_finetuned_int8': {'parseable_rate': 0.93,
  'exact_match_rate': 0.13,
  'lat_mean': 2.671546212150024,
  'lat_p50': 3.454315853500096,
  'lat_p95': 3.6232871275001344}}

**15. Improve SQL-only decoding + log results to MLflow + show table**

In [26]:
import pandas as pd
import mlflow

STRICT_SYSTEM = (
    "You are an expert PostgreSQL query generator. "
    "Output ONLY a single valid PostgreSQL query. "
    "No explanations, no markdown, no code fences, no extra text."
)

@torch.inference_mode()
def batched_generate_strict(model, prompts, batch_size=BATCH_SIZE_EVAL, max_new_tokens=MAX_NEW_TOKENS):
    model.eval()
    latencies = []
    outputs = []

    for i in tqdm(range(0, len(prompts), batch_size), desc="Batched inference (strict)"):
        batch_prompts = prompts[i:i+batch_size]

        batch_texts = []
        for p in batch_prompts:
            msgs = [
                {"role":"system", "content": STRICT_SYSTEM},
                {"role":"user", "content": p},
            ]
            batch_texts.append(
                tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
            )

        enc = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SEQ_LEN,
        ).to(model.device)

        start = time.perf_counter()
        gen = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        end = time.perf_counter()

        batch_time = end - start
        per_ex = batch_time / len(batch_prompts)
        latencies.extend([per_ex] * len(batch_prompts))

        for j in range(len(batch_prompts)):
            prompt_len = enc["input_ids"][j].shape[0]
            decoded = tokenizer.decode(gen[j][prompt_len:], skip_special_tokens=True)
            outputs.append(strip_code_fences(decoded))

    return outputs, latencies

# Re-evaluate only finetuned models with strict prompting to recover parseability
pred_ft2, lat_ft2 = batched_generate_strict(finetuned_fp16, test_prompts)
m_ft2 = eval_predictions(pred_ft2, test_gold)
local_results["qwen_finetuned_fp16_strict"] = {**m_ft2, **{f"lat_{k}": v for k, v in latency_stats(lat_ft2).items()}}

pred_i82, lat_i82 = batched_generate_strict(finetuned_int8, test_prompts)
m_i82 = eval_predictions(pred_i82, test_gold)
local_results["qwen_finetuned_int8_strict"] = {**m_i82, **{f"lat_{k}": v for k, v in latency_stats(lat_i82).items()}}

# Log to MLflow under the SAME parent run as a new nested run
with mlflow.start_run(run_name=PARENT_RUN_NAME):
    with mlflow.start_run(run_name="eval_local", nested=True):
        for model_name, metrics in local_results.items():
            for k, v in metrics.items():
                if v is None:
                    continue
                mlflow.log_metric(f"{model_name}.{k}", float(v))

# Display table
df_local = pd.DataFrame(local_results).T.reset_index().rename(columns={"index":"model"})
df_local

Batched inference (strict):   0%|          | 0/9 [00:00<?, ?it/s]

Batched inference (strict):   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run eval_local at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/61fe016316a94891bd5eb6b4d6278843
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1
🏃 View run colab_run_20260206_085921 at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/1ebc4c37c7804263b0e9da380caa1237
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1


,model,parseable_rate,exact_match_rate,lat_mean,lat_p50,lat_p95
0,qwen_baseline_fp16,1.00,0.09,0.405469,0.422399,0.623889
1,qwen_finetuned_fp16,0.93,0.13,0.526728,0.710791,0.739464
2,qwen_finetuned_int8,0.93,0.13,2.671546,3.454316,3.623287
3,qwen_finetuned_fp16_strict,1.00,0.15,0.432624,0.426541,0.735788
4,qwen_finetuned_int8_strict,0.99,0.20,2.151604,2.541377,3.609619


**16: Model Evaluations using API (GPT-4o-mini, Claude 3.5 Haiku)**

In [31]:
import os, time
import pandas as pd
import mlflow
from tqdm import tqdm

# --- OpenAI ---
from openai import OpenAI
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# --- Anthropic ---
import anthropic
anthropic_client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

API_SYSTEM = STRICT_SYSTEM  # from Cell 15

def safe_sleep(attempt: int):
    time.sleep(min(2.0 * attempt, 6.0))

def eval_openai(prompts, gold, model_name="gpt-4o-mini", max_tokens=MAX_NEW_TOKENS):
    preds, lat = [], []
    prompt_toks, completion_toks = 0, 0

    for p in tqdm(prompts, desc=f"API eval: {model_name}"):
        for attempt in range(1, 4):
            try:
                t0 = time.perf_counter()
                resp = openai_client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": API_SYSTEM},
                        {"role": "user", "content": p},
                    ],
                    temperature=0.0,
                    max_tokens=max_tokens,
                )
                t1 = time.perf_counter()
                lat.append(t1 - t0)

                text = (resp.choices[0].message.content or "").strip()
                preds.append(strip_code_fences(text))

                if getattr(resp, "usage", None):
                    prompt_toks += int(resp.usage.prompt_tokens or 0)
                    completion_toks += int(resp.usage.completion_tokens or 0)
                break
            except Exception:
                if attempt == 3:
                    preds.append("")
                    lat.append(None)
                else:
                    safe_sleep(attempt)

    lat_clean = [x for x in lat if x is not None]
    m = eval_predictions(preds, gold)
    m.update({f"lat_{k}": v for k, v in latency_stats(lat_clean).items()})
    m.update({"prompt_tokens": prompt_toks, "completion_tokens": completion_toks})
    return preds, m

def eval_anthropic(prompts, gold, model_name="claude-3-5-haiku-latest", max_tokens=MAX_NEW_TOKENS):
    preds, lat = [], []
    in_toks, out_toks = 0, 0

    for p in tqdm(prompts, desc=f"API eval: {model_name}"):
        for attempt in range(1, 4):
            try:
                t0 = time.perf_counter()
                resp = anthropic_client.messages.create(
                    model=model_name,
                    system=API_SYSTEM,
                    messages=[{"role": "user", "content": p}],
                    temperature=0.0,
                    max_tokens=max_tokens,
                )
                t1 = time.perf_counter()
                lat.append(t1 - t0)

                # Join text blocks
                text = "".join([blk.text for blk in resp.content if hasattr(blk, "text")]) if resp.content else ""
                preds.append(strip_code_fences(text))

                if getattr(resp, "usage", None):
                    in_toks += int(getattr(resp.usage, "input_tokens", 0) or 0)
                    out_toks += int(getattr(resp.usage, "output_tokens", 0) or 0)
                break
            except Exception:
                if attempt == 3:
                    preds.append("")
                    lat.append(None)
                else:
                    safe_sleep(attempt)

    lat_clean = [x for x in lat if x is not None]
    m = eval_predictions(preds, gold)
    m.update({f"lat_{k}": v for k, v in latency_stats(lat_clean).items()})
    m.update({"input_tokens": in_toks, "output_tokens": out_toks})
    return preds, m

# Run API eval on the SAME prompts you used locally (already strict formatted prompts)
pred_oai, metrics_oai = eval_openai(test_prompts, test_gold, model_name="gpt-4o-mini")
pred_haiku, metrics_haiku = eval_anthropic(test_prompts, test_gold, model_name="claude-3-5-haiku-latest")

api_results = {
    "gpt-4o-mini": metrics_oai,
    "claude-3.5-haiku": metrics_haiku,
}

# Log to MLflow
with mlflow.start_run(run_name=PARENT_RUN_NAME):
    with mlflow.start_run(run_name="eval_api", nested=True):
        for model_name, metrics in api_results.items():
            for k, v in metrics.items():
                if v is None:
                    continue
                mlflow.log_metric(f"{model_name}.{k}", float(v))

# Combine results into a single notebook table
all_results = {**local_results, **api_results}
df_all = pd.DataFrame(all_results).T.reset_index().rename(columns={"index": "model"})
df_all

API eval: claude-3-5-haiku-latest:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipython-input-2450873966.py:67: DeprecationWarning: The model 'claude-3-5-haiku-latest' is deprecated and will reach end-of-life on February 19th, 2026.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  resp = anthropic_client.messages.create(
API eval: claude-3-5-haiku-latest: 100%|██████████| 100/100 [02:53<00:00,  1.74s/it]


🏃 View run eval_api at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/2ef9d5d8453f40a4af5563b2861e7d4a
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1
🏃 View run colab_run_20260206_085921 at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1/runs/fc7c10559ef94ada9772431bb8eb4ed1
🧪 View experiment at: https://dagshub.com/aravula7/llm-finetuning.mlflow/#/experiments/1


,model,parseable_rate,exact_match_rate,lat_mean,lat_p50,lat_p95,prompt_tokens,completion_tokens,input_tokens,output_tokens
0,qwen_baseline_fp16,1.00,0.09,0.405469,0.422399,0.623889,NaN,NaN,NaN,NaN
1,qwen_finetuned_fp16,0.93,0.13,0.526728,0.710791,0.739464,NaN,NaN,NaN,NaN
2,qwen_finetuned_int8,0.93,0.13,2.671546,3.454316,3.623287,NaN,NaN,NaN,NaN
3,qwen_finetuned_fp16_strict,1.00,0.15,0.432624,0.426541,0.735788,NaN,NaN,NaN,NaN
4,qwen_finetuned_int8_strict,0.99,0.20,2.151604,2.541377,3.609619,NaN,NaN,NaN,NaN
5,gpt-4o-mini,1.00,0.04,1.615721,1.551064,2.820497,12511.0,5449.0,NaN,NaN
6,claude-3.5-haiku,0.99,0.07,1.734534,1.540677,2.696610,NaN,NaN,14251.0,8465.0
